# Computing chemical properties

So far, we have gained an understanding of how to work with [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) and [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html). Below, we continue learning to evaluate *thermodynamic and chemical properties of the system, its phases, and constituting species* using class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html).

Let's start with a simple chemical system containing a single aqueous phase and chemical state representing a 1 molal NaCl solution with 0.7 molal dissolved CO<sub>2</sub>:

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))
# HINT: solution.setActivityModel(ActivityModelPitzerHMW())

system = ChemicalSystem(db, solution)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 0.7, "mol")

equilibrate(state)

print(state)

> **Note**: Above, we use a convenience method [equilibrate](https://reaktoro.org/api/namespaceReaktoro.html#a286e2972325304934d631234abef9e87) to quickly bring the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object `state` into a state of chemical equilibrium. It is implemented for the case when the Gibbs energy minimization has to be performed once. However, for the *sequence of chemical equilibrium calculations*, we recommend using [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) class, which
> * improves the performance (avoiding initialization of new equilibrium solver at each call of `equilibrate`)
> * add flexibility to enforce different equilibrium constraints at equilibrium (e.g., fixed pH, pE, Eh, gas fugacity, enthalpy, internal energy, volume, etc.) in combination classes [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) and [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html).

Generally speaking, computing chemical properties of [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) objects is a task for class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html).

In [ ]:
props = ChemicalProps(state)
print(props)

> **Note**: `ActivityCoefficients` above are all set to 1.0, since all phases in chemical system are assigned with **ideal activity models** by default.

For comparison, we reconstruct the [ChemicalSystem](https://reaktoro.org/api/classReaktoro_1_1ChemicalSystem.html) with *Pitzer's activity model* for aqueous phase and activity model based on *Peng-Robinson's equation of state* for the gaseous phase as well as reevaluate chemical state and its properties:

In [ ]:
solution = AqueousPhase(speciate("H O Na Cl C Ca Si"))
solution.setActivityModel(ActivityModelPitzerHMW())

gases = GaseousPhase()
gases.setActivityModel(ActivityModelPengRobinson())

minerals = MineralPhases()

system = ChemicalSystem(db, solution, gases, minerals)

state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.pressure(15.0, "atm")
state.set("H2O"    , 1.0, "kg")
state.set("Na+"    , 1.0, "mol")
state.set("Cl-"    , 1.0, "mol")
state.set("CO2"    , 0.7, "mol")
state.set("Calcite", 1.0, "g")
state.set("Quartz" , 1.0, "g")

equilibrate(state)

# Chemical properties can be also fetched from the chemical state as they get updated at the end of the optimization algorithm
props = state.props()

print(props)

The results displayed above should now be a little more interesting.

> **Note**: chemical properties can be also accessed by `state.props()` as they are being evaluated at the end of the minimization algorithm run in the `equilibrate()` method.

Now consider a scenario where the chemical state has changed, for example, 1 mol of NaCl has been added to it and the temperature has increased by 10 °C. The chemical properties of the system must then be updated as shown below:

In [ ]:
state.temperature(70.0, "celsius")
state.add("Na+", 1.0, "mol")
state.add("Cl-", 1.0, "mol")

props.update(state)

print(props)

The printed table above shows a long list of properties. Below, we use some methods from class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) to retrieve specific properties for the system, its phases, and its species:

In [ ]:
print(f"SYSTEM'S TEMPERATURE     : {props.temperature()                        } K    ")
print(f"AQUEOUS PHASE'S DENSITY  : {props.phaseProps('AqueousPhase').density() } kg/m3")
print(f"AQUEOUS PHASE'S ENTHALPY : {props.phaseProps('AqueousPhase').enthalpy()} J/mol")
print(f"GASEOUS PHASE'S DENSITY  : {props.phaseProps('GaseousPhase').density() } kg/m3")
print(f"GASEOUS PHASE'S ENTHALPY : {props.phaseProps('GaseousPhase').enthalpy()} J/mol")
print(f"ACTIVITY H+              : {props.speciesActivity('H+')                }      ")
print(f"ACTIVITY H+ (LOG10)      : {props.speciesActivityLg('H+')              }      ")
print(f"ACTIVITY COEFF. CO2      : {props.speciesActivityCoefficient('CO2')    }      ")
print(f"FUGACITY COEFF. CO2(g)   : {props.speciesActivityCoefficient('CO2(g)') }      ")

**TASK**: using [API of the ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) class, print out the following properties of the system:
* pressure and volume
* aqueous phase's mass and volume
* aqueous phase's mass and volume
* activity coefficient of species `HCO3-`
* fugacity coefficient of `H2O(g)` and `O2(g)` gases

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #